In [1]:
import pandas as pd
import numpy as np

In [ ]:
games = pd.read_csv('games.csv')

In [ ]:
games.columns

In [ ]:
refDf = games.groupby('referee').size()
refDf

In [ ]:
lst = []
for i in refDf.index:
    lst.append((i, games[games['referee'] == i]['spread_line'].sum(), games[games['referee'] == i]['game_id'].count()))

spreadDf = pd.DataFrame(lst, columns= ['referee','total_spread','total_games'])
spreadDf['ref_spread'] = spreadDf['total_spread']/spreadDf['total_games']

refJoin= spreadDf.drop(['total_spread', 'total_games'], axis=1)

In [ ]:
games = games.merge(refJoin, on='referee')

In [ ]:
qb_df = games[['game_id', 'result', 'home_qb_name', 'away_qb_name']]
qb_df = qb_df.dropna()

In [ ]:
# Calculate total_result and total_games
qb_results = games.groupby('home_qb_name')['result'].sum().reset_index(name='total_result')
qb_games = games.groupby('home_qb_name')['game_id'].count().reset_index(name='total_games')

# Calculate home_qb_result
qb_results['home_qb_result'] = qb_results['total_result'] / qb_games['total_games']

# Merge aggregated results back to games DataFrame
games = games.merge(qb_results[['home_qb_name', 'home_qb_result']], on='home_qb_name', how='left')


In [ ]:
# Calculate total_result and total_games for away quarterbacks
away_qb_results = games.groupby('away_qb_name')['result'].sum().reset_index(name='total_result')
away_qb_results['total_result'] *= -1  # Multiply the sum by -1

away_qb_games = games.groupby('away_qb_name')['game_id'].count().reset_index(name='total_games')

# Calculate away_qb_result
away_qb_results['away_qb_result'] = away_qb_results['total_result'] / away_qb_games['total_games']

# Merge aggregated results back to games DataFrame
games = games.merge(away_qb_results[['away_qb_name', 'away_qb_result']], on='away_qb_name', how='left')


In [ ]:
games.tail(5)

In [ ]:
# Calculate total_result and total_games
coach_results = games.groupby('home_coach')['result'].sum().reset_index(name='total_result')
coach_games = games.groupby('home_coach')['game_id'].count().reset_index(name='total_games')

# Calculate home_qb_result
coach_results['home_coach_result'] = coach_results['total_result'] / coach_games['total_games']

# Merge aggregated results back to games DataFrame
games = games.merge(coach_results[['home_coach', 'home_coach_result']], on='home_coach', how='left')

In [ ]:
# Calculate total_result and total_games
coach_results = games.groupby('away_coach')['result'].sum().reset_index(name='total_result')
coach_games = games.groupby('away_coach')['game_id'].count().reset_index(name='total_games')

# Calculate home_qb_result
coach_results['away_coach_result'] = coach_results['total_result']*-1 / coach_games['total_games']

# Merge aggregated results back to games DataFrame
games = games.merge(coach_results[['away_coach', 'away_coach_result']], on='away_coach', how='left')

Preprocessing steps:
1. Group and get counts of QBs, refs, coaches
2. Find total spread for each grouping 
3. Find total outcome
4. Average findings

In [ ]:
games.to_csv('gamesRating.csv')

In [73]:
stadiums = pd.read_csv('NFL Stadium Latitude and Longtitude.csv')
teams = pd.read_csv('teams.csv')
games = pd.read_csv('gamesRating.csv')

In [74]:
teams.head()

,season,team,nfl,nfl_team_id,espn,pfr,pff,pfflabel,fo,full,location,short_location,nickname,hyphenated,sbr,sbr_wins,sbr_name,draft_kings
0,2002,ARI,ARI,3800,ARI,CRD,NaN,NaN,ARI,Arizona Cardinals,Arizona,Arizona,Cardinals,arizona-cardinals,1549,NaN,NaN,NaN
1,2002,ATL,ATL,200,ATL,ATL,NaN,NaN,ATL,Atlanta Falcons,Atlanta,Atlanta,Falcons,atlanta-falcons,1546,NaN,NaN,NaN
2,2002,BAL,BAL,325,BAL,RAV,NaN,NaN,BAL,Baltimore Ravens,Baltimore,Baltimore,Ravens,baltimore-ravens,1521,NaN,NaN,NaN
3,2002,BUF,BUF,610,BUF,BUF,NaN,NaN,BUF,Buffalo Bills,Buffalo,Buffalo,Bills,buffalo-bills,1526,NaN,NaN,NaN
4,2002,CAR,CAR,750,CAR,CAR,NaN,NaN,CAR,Carolina Panthers,Carolina,Carolina,Panthers,carolina-panthers,1545,NaN,NaN,NaN


In [75]:
stadiums = stadiums.rename(mapper={'Team':'nickname'}, axis=1)
stadiums

,nickname,Conference,latitude,longitude,zip
0,Titans,AFC,36.166461,-86.771289,37213
1,Giants,NFC,40.812194,-74.076983,10021
2,Steelers,AFC,40.446786,-80.015761,15202
3,Panthers,NFC,35.225808,-80.852861,28202
4,Ravens,AFC,39.277969,-76.622767,21203
5,Buccaneers,NFC,27.975967,-82.503350,33607
6,Colts,AFC,39.760056,-86.163806,46201
7,Vikings,NFC,44.973881,-93.258094,56760
8,Cardinals,NFC,33.527700,-112.262608,63102
9,Cowboys,NFC,32.747778,-97.092778,75203


In [76]:
stadiums = stadiums.merge(teams.groupby('nickname').first().reset_index()[['nickname','team']], on='nickname', how='left')

In [77]:
stadiums = stadiums.rename(mapper={'team':'home_team'}, axis=1)
games.head()

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,away_coach,home_coach,referee,stadium_id,stadium,ref_spread,home_qb_result,away_qb_result,home_coach_result,away_coach_result
0,1999_01_MIN_ATL,1999,REG,1,9/12/99,Sunday,NaN,MIN,17,ATL,...,Dennis Green,Dan Reeves,Gerry Austin,ATL00,Georgia Dome,2.158915,-2.666667,0.428571,-3.128205,-7.632653
1,1999_02_PIT_BAL,1999,REG,2,9/19/99,Sunday,NaN,PIT,23,BAL,...,Bill Cowher,Brian Billick,Gerry Austin,BAL00,PSINet Stadium,2.158915,3.666667,0.821429,7.173333,3.161765
2,1999_04_TEN_SF,1999,REG,4,10/3/99,Sunday,NaN,TEN,22,SF,...,Jeff Fisher,Steve Mariucci,Gerry Austin,SFO00,3Com Park,2.158915,4.688525,-0.400000,0.745455,-2.894366
3,1999_05_MIA_IND,1999,REG,5,10/10/99,Sunday,NaN,MIA,34,IND,...,Jimmy Johnson,Jim Mora,Gerry Austin,IND99,RCA Dome,2.158915,8.700000,-8.750000,4.603448,-8.300000
4,1999_06_WAS_ARI,1999,REG,6,10/17/99,Sunday,NaN,WAS,24,ARI,...,Norv Turner,Vince Tobin,Gerry Austin,PHO99,Sun Devil Stadium,2.158915,1.087719,0.923077,-6.071429,-0.766234


In [78]:
stadiums.loc[23, 'home_team'] = 'SF'

In [47]:
games.to_csv('gamesMerged.csv', index=False)

In [31]:
weather = pd.read_csv('open-meteo-36.17N86.73W122m.csv', low_memory=False)

In [32]:
weather['timeDate'] = pd.to_datetime(weather['time'])

# Create a new column with the desired date format
weather['dateFormat'] = weather['timeDate'].dt.strftime('%m/%d/%y')


In [33]:
weather.head()

,location_id,time,temperature_2m (°C),precipitation (mm),timeDate,dateFormat
0,0,2000-01-01T00:00,11.0,0.0,2000-01-01 00:00:00,01/01/00
1,0,2000-01-01T01:00,10.0,0.0,2000-01-01 01:00:00,01/01/00
2,0,2000-01-01T02:00,9.2,0.0,2000-01-01 02:00:00,01/01/00
3,0,2000-01-01T03:00,8.5,0.0,2000-01-01 03:00:00,01/01/00
4,0,2000-01-01T04:00,8.5,0.0,2000-01-01 04:00:00,01/01/00


In [34]:
weather['time'] = weather['timeDate'].dt.strftime('%H:%M')

In [35]:
weather = weather.drop('timeDate', axis=1)

In [36]:
weather.to_csv('weather.csv', index=False)

In [37]:
len(weather)

6686208

In [79]:
location_ids = pd.read_csv('locationids.csv')

In [80]:
stadiums

,nickname,Conference,latitude,longitude,zip,home_team
0,Titans,AFC,36.166461,-86.771289,37213,TEN
1,Giants,NFC,40.812194,-74.076983,10021,NYG
2,Steelers,AFC,40.446786,-80.015761,15202,PIT
3,Panthers,NFC,35.225808,-80.852861,28202,CAR
4,Ravens,AFC,39.277969,-76.622767,21203,BAL
5,Buccaneers,NFC,27.975967,-82.503350,33607,TB
6,Colts,AFC,39.760056,-86.163806,46201,IND
7,Vikings,NFC,44.973881,-93.258094,56760,MIN
8,Cardinals,NFC,33.527700,-112.262608,63102,ARI
9,Cowboys,NFC,32.747778,-97.092778,75203,DAL


In [81]:
stadiums = stadiums.merge(location_ids[['location_id']], how='outer', left_index=True, right_index=True) 

In [67]:
games = games.drop('location_id', axis=1)

In [82]:
games = games.merge(stadiums[['location_id', 'home_team']], on='home_team', how='left')

In [83]:
games

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,home_coach,referee,stadium_id,stadium,ref_spread,home_qb_result,away_qb_result,home_coach_result,away_coach_result,location_id
0,1999_01_MIN_ATL,1999,REG,1,9/12/99,Sunday,NaN,MIN,17,ATL,...,Dan Reeves,Gerry Austin,ATL00,Georgia Dome,2.158915,-2.666667,0.428571,-3.128205,-7.632653,10.0
1,1999_02_PIT_BAL,1999,REG,2,9/19/99,Sunday,NaN,PIT,23,BAL,...,Brian Billick,Gerry Austin,BAL00,PSINet Stadium,2.158915,3.666667,0.821429,7.173333,3.161765,4.0
2,1999_04_TEN_SF,1999,REG,4,10/3/99,Sunday,NaN,TEN,22,SF,...,Steve Mariucci,Gerry Austin,SFO00,3Com Park,2.158915,4.688525,-0.400000,0.745455,-2.894366,23.0
3,1999_05_MIA_IND,1999,REG,5,10/10/99,Sunday,NaN,MIA,34,IND,...,Jim Mora,Gerry Austin,IND99,RCA Dome,2.158915,8.700000,-8.750000,4.603448,-8.300000,6.0
4,1999_06_WAS_ARI,1999,REG,6,10/17/99,Sunday,NaN,WAS,24,ARI,...,Vince Tobin,Gerry Austin,PHO99,Sun Devil Stadium,2.158915,1.087719,0.923077,-6.071429,-0.766234,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6448,2022_18_DAL_WAS,2022,REG,18,1/8/23,Sunday,16:25,DAL,6,WAS,...,Ron Rivera,Tra Blake,WAS00,FedExField,2.861111,12.000000,3.560000,1.728155,1.042857,17.0
6449,2023_01_HOU_BAL,2023,REG,1,9/10/23,Sunday,13:00,HOU,9,BAL,...,John Harbaugh,Tra Blake,BAL00,M&T Bank Stadium,2.861111,6.294118,-16.000000,7.650794,-16.000000,4.0
6450,2023_02_GB_ATL,2023,REG,2,9/17/23,Sunday,13:00,GB,24,ATL,...,Arthur Smith,Tra Blake,ATL97,Mercedes-Benz Stadium,2.861111,7.250000,3.666667,-1.473684,0.027778,10.0
6451,2023_01_ARI_WAS,2023,REG,1,9/10/23,Sunday,13:00,ARI,16,WAS,...,Ron Rivera,Alan Eck,WAS00,FedExField,2.250000,12.000000,-4.000000,1.728155,-4.000000,17.0


In [84]:
games.to_csv('gamesMerged.csv', index=False)